In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import json
EXPERIENCE_PATH = './trainingdata/hansolo/young-han.json'
PROTECTED_PATH = './trainingdata/hansolo/young-han-protective.json'

new_experience = []
new_protected = []

# loop thru the original jsons
# for each object present, do the following:
# 1. create an empty dict
# 2. add the key "text"

# with open(EXPERIENCE_PATH, 'r') as f:
#     exp = json.load(f)
#     for obj in exp:
#         #print(obj)
#         temp = {'text': []}
#         temp['text'].append('<instruction>')
#         temp['text'].append(obj['instruction'])
#         temp['text'].append('<input>')
#         temp['text'].append(obj['input'])
#         temp['text'].append('<output>')
#         temp['text'].append(obj['output'])
#         temp['text'] = ' '.join(temp['text'])
#         new_experience.append(temp)
#     f.close()
# print(len(new_experience))
# # create a json file with the new experience
# with open('./trainingdata/hansolo/young-han-new.jsonl', 'w') as f:
#     for entry in new_experience:
#         json.dump(entry, f)
#         f.write('\n')
#     # json.dump(new_experience, f)
#     f.close()

with open(PROTECTED_PATH, 'r') as f:
    exp = json.load(f)
    for obj in exp:
        #print(obj)
        temp = {'text': []}
        temp['text'].append('<instruction>')
        temp['text'].append(obj['instruction'])
        temp['text'].append('<input>')
        temp['text'].append(obj['input'])
        temp['text'].append('<output>')
        temp['text'].append(obj['output'])
        temp['text'] = ' '.join(temp['text'])
        new_protected.append(temp)
    f.close()

with open('./trainingdata/hansolo/young-han-protective-new.jsonl', 'w') as f:
    for entry in new_protected:
        json.dump(entry, f)
        f.write('\n')
    # json.dump(new_experience, f)
    f.close()


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
import together
from apikey import KEY

EXPERIENCE_PATH = './trainingdata/hansolo/young-han-new.jsonl'
# PROTECTED_PATH = './trainingdata/hansolo/young-han-protective-new.jsonl'

together.api_key = KEY

model_list = together.Models.list()

print(f"{len(model_list)} models available")

# print the first 10 models on the menu
# model_names = [model_dict['name'] for model_dict in model_list]
# print(model_names[:10])

105 models available


In [38]:
resp = together.Files.upload(file=EXPERIENCE_PATH)
file_id = resp["id"]

Uploading ./trainingdata/hansolo/young-han-new.jsonl: 100%|██████████| 781k/781k [00:00<00:00, 1.92MB/s]


In [6]:
files_list = together.Files.list()
files_list['data']

[{'filename': 'young-han-new.jsonl',
  'bytes': 799730,
  'created_at': 1699945068,
  'id': 'file-bb280f38-2b64-42a9-9349-6e7eb2ad2d05',
  'purpose': 'fine-tune',
  'object': 'file',
  'LineCount': 0,
  'Processed': True}]

In [8]:
resp = together.Finetune.create(
  training_file = 'file-bb280f38-2b64-42a9-9349-6e7eb2ad2d05',
  model = 'togethercomputer/llama-2-70b-chat',
  n_epochs = 3,
  n_checkpoints = 1,
  batch_size = 4,
  learning_rate = 1e-5,
  suffix = 'my-demo-finetune',
  #wandb_api_key = KEY,
)

fine_tune_id = resp['id']
print(resp)

{'training_file': 'file-bb280f38-2b64-42a9-9349-6e7eb2ad2d05', 'validation_file': '', 'model_output_name': 'mifu67@stanford.edu/llama-2-70b-chat-my-demo-finetune-2023-11-17-20-51-35', 'model_output_path': 's3://together-dev/finetune/6552b4a556bb2d3952ed7a14/mifu67@stanford.edu/llama-2-70b-chat-my-demo-finetune-2023-11-17-20-51-35/ft-3625edb3-0c2d-4cf7-9164-e45532dfea2b', 'Suffix': 'my-demo-finetune', 'model': 'togethercomputer/llama-2-70b-chat', 'n_epochs': 3, 'n_checkpoints': 1, 'batch_size': 32, 'learning_rate': 1e-05, 'user_id': '6552b4a556bb2d3952ed7a14', 'lora': False, 'lora_r': 8, 'lora_alpha': 8, 'lora_dropout': 0, 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-11-17T20:51:35.538Z', 'updated_at': '2023-11-17T20:51:35.538Z', 'status': 'pending', 'owner_address': '0xebd597774b313d8e8d22af4629b75ec7d567f678', 'id': 'ft-3625edb3-0c2d-4cf7-9164-e45532dfea2b', 'job_id': '', 'token_count': 0, 'param_count': 0, 'total_price': 0, 

In [19]:
resp = together.Finetune.list()
resp['data'][-2]['status']
# print(len(resp['data']))
# print(resp['data'][3]['id'])
# resp['data'][4]['status'] 

'queued'

In [13]:
together.Models.start("mifu67@stanford.edu/llama-2-7b-chat-my-demo-finetune-2023-11-17-20-51-22")

{'success': True,
 'value': '4e32ecb6957facee7bdb68237575eda13590c5ac55a669cea78dbc4a56d583d9-51a09629248aaf3729e00e43e6c0d2554f542a5329a86f366bac2fc4e71205f0',
 'wasAlreadyEnabled': True}

In [17]:
output = together.Complete.create(
  prompt = "What are your thoughts on Count Dooku?\n\n1. ", 
  model = "mifu67@stanford.edu/llama-2-7b-chat-my-demo-finetune-2023-11-17-20-51-22", 
)

In [18]:
output

{'status': 'finished',
 'prompt': ['What are your thoughts on Count Dooku?\n\n1. '],
 'model': 'mifu67@stanford.edu/llama-2-7b-chat-my-demo-finetune-2023-11-17-20-51-22',
 'model_owner': '',
 'tags': {},
 'num_returns': 1,
 'args': {'model': 'mifu67@stanford.edu/llama-2-7b-chat-my-demo-finetune-2023-11-17-20-51-22',
  'prompt': 'What are your thoughts on Count Dooku?\n\n1. ',
  'top_p': 0.7,
  'top_k': 50,
  'temperature': 0.7,
  'max_tokens': 128,
  'stop': [],
  'repetition_penalty': None,
  'logprobs': None},
 'subjobs': [],
 'output': {'choices': [{'text': 'He is a compelling and complex character.\n2.  He is a formidable warrior and strategist.\n3.  He has a tragic backstory that adds depth to his character.\n4.  He is a loyal servant of the Sith, but also has his own motivations and desires.\n5.  He is a skilled duelist, able to hold his own against even the most powerful opponents.\n6.  He has a distinctive and intimidating appearance, with his red lightsaber and mask.\n7.  He i